# Stock purchase recommendations with Machine Learning

In [30]:
import pandas as pd
import numpy as np
import talib as ta
import matplotlib.pyplot as plt
from tqdm import tqdm, tqdm_notebook # progress bar
import fastparquet
import pickle

from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import fbeta_score
from sklearn.metrics import make_scorer
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import StandardScaler

from sklearn.ensemble import RandomForestClassifier

In [7]:
pd.set_option('display.max_columns', 1500)

## 1) Load Training and Test Data

In [8]:
# load the training and test datae from feature engineering step:
X_train = fastparquet.ParquetFile('../data/processed/X_train.parq').to_pandas()
X_test = fastparquet.ParquetFile('../data/processed/X_test.parq').to_pandas()
y_train = pickle.load(open('../data/processed/y_train.pkl', 'rb'))
y_test = pickle.load(open('../data/processed/y_test.pkl', 'rb'))

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((7024, 693), (1756, 693), (7024,), (1756,))

In [9]:
# quick inspection
X_train.tail()

,AdjVolume_-19,AdjVolume_-18,AdjVolume_-17,AdjVolume_-16,AdjVolume_-15,AdjVolume_-14,AdjVolume_-13,AdjVolume_-12,AdjVolume_-11,AdjVolume_-10,AdjVolume_-9,AdjVolume_-8,AdjVolume_-7,AdjVolume_-6,AdjVolume_-5,AdjVolume_-4,AdjVolume_-3,AdjVolume_-2,AdjVolume_-1,AdjVolume_0,AdjOpen_-19,AdjOpen_-18,AdjOpen_-17,AdjOpen_-16,AdjOpen_-15,AdjOpen_-14,AdjOpen_-13,AdjOpen_-12,AdjOpen_-11,AdjOpen_-10,AdjOpen_-9,AdjOpen_-8,AdjOpen_-7,AdjOpen_-6,AdjOpen_-5,AdjOpen_-4,AdjOpen_-3,AdjOpen_-2,AdjOpen_-1,AdjOpen_0,AdjHigh_-19,AdjHigh_-18,AdjHigh_-17,AdjHigh_-16,AdjHigh_-15,AdjHigh_-14,AdjHigh_-13,AdjHigh_-12,AdjHigh_-11,AdjHigh_-10,AdjHigh_-9,AdjHigh_-8,AdjHigh_-7,AdjHigh_-6,AdjHigh_-5,AdjHigh_-4,AdjHigh_-3,AdjHigh_-2,AdjHigh_-1,AdjHigh_0,AdjLow_-19,AdjLow_-18,AdjLow_-17,AdjLow_-16,AdjLow_-15,AdjLow_-14,AdjLow_-13,AdjLow_-12,AdjLow_-11,AdjLow_-10,AdjLow_-9,AdjLow_-8,AdjLow_-7,AdjLow_-6,AdjLow_-5,AdjLow_-4,AdjLow_-3,AdjLow_-2,AdjLow_-1,AdjLow_0,AdjClose_-19,AdjClose_-18,AdjClose_-17,AdjClose_-16,AdjClose_-15,AdjClose_-14,AdjClose_-13,AdjClose_-12,AdjClose_-11,AdjClose_-10,AdjClose_-9,AdjClose_-8,AdjClose_-7,AdjClose_-6,AdjClose_-5,AdjClose_-4,AdjClose_-3,AdjClose_-2,AdjClose_-1,AdjClose_0,AdjCloseRSI14_-19,AdjCloseRSI14_-18,AdjCloseRSI14_-17,AdjCloseRSI14_-16,AdjCloseRSI14_-15,AdjCloseRSI14_-14,AdjCloseRSI14_-13,AdjCloseRSI14_-12,AdjCloseRSI14_-11,AdjCloseRSI14_-10,AdjCloseRSI14_-9,AdjCloseRSI14_-8,AdjCloseRSI14_-7,AdjCloseRSI14_-6,AdjCloseRSI14_-5,AdjCloseRSI14_-4,AdjCloseRSI14_-3,AdjCloseRSI14_-2,AdjCloseRSI14_-1,AdjCloseRSI14_0,RSI_above_80_-19,RSI_above_80_-18,RSI_above_80_-17,RSI_above_80_-16,RSI_above_80_-15,RSI_above_80_-14,RSI_above_80_-13,RSI_above_80_-12,RSI_above_80_-11,RSI_above_80_-10,RSI_above_80_-9,RSI_above_80_-8,RSI_above_80_-7,RSI_above_80_-6,RSI_above_80_-5,RSI_above_80_-4,RSI_above_80_-3,RSI_above_80_-2,RSI_above_80_-1,RSI_above_80_0,RSI_below_20_-19,RSI_below_20_-18,RSI_below_20_-17,RSI_below_20_-16,RSI_below_20_-15,RSI_below_20_-14,RSI_below_20_-13,RSI_below_20_-12,RSI_below_20_-11,RSI_below_20_-10,RSI_below_20_-9,RSI_below_20_-8,RSI_below_20_-7,RSI_below_20_-6,RSI_below_20_-5,RSI_below_20_-4,RSI_below_20_-3,RSI_below_20_-2,RSI_below_20_-1,RSI_below_20_0,AdjCloseSMA10_-19,AdjCloseSMA10_-18,AdjCloseSMA10_-17,AdjCloseSMA10_-16,AdjCloseSMA10_-15,AdjCloseSMA10_-14,AdjCloseSMA10_-13,AdjCloseSMA10_-12,AdjCloseSMA10_-11,AdjCloseSMA10_-10,AdjCloseSMA10_-9,AdjCloseSMA10_-8,AdjCloseSMA10_-7,AdjCloseSMA10_-6,AdjCloseSMA10_-5,AdjCloseSMA10_-4,AdjCloseSMA10_-3,AdjCloseSMA10_-2,AdjCloseSMA10_-1,AdjCloseSMA10_0,AdjCloseSMA10_to_AdjClose_ratio_-19,AdjCloseSMA10_to_AdjClose_ratio_-18,AdjCloseSMA10_to_AdjClose_ratio_-17,AdjCloseSMA10_to_AdjClose_ratio_-16,AdjCloseSMA10_to_AdjClose_ratio_-15,AdjCloseSMA10_to_AdjClose_ratio_-14,AdjCloseSMA10_to_AdjClose_ratio_-13,AdjCloseSMA10_to_AdjClose_ratio_-12,AdjCloseSMA10_to_AdjClose_ratio_-11,AdjCloseSMA10_to_AdjClose_ratio_-10,AdjCloseSMA10_to_AdjClose_ratio_-9,AdjCloseSMA10_to_AdjClose_ratio_-8,AdjCloseSMA10_to_AdjClose_ratio_-7,AdjCloseSMA10_to_AdjClose_ratio_-6,AdjCloseSMA10_to_AdjClose_ratio_-5,AdjCloseSMA10_to_AdjClose_ratio_-4,AdjCloseSMA10_to_AdjClose_ratio_-3,AdjCloseSMA10_to_AdjClose_ratio_-2,AdjCloseSMA10_to_AdjClose_ratio_-1,AdjCloseSMA10_to_AdjClose_ratio_0,AdjClose_above_AdjCloseSMA10_-19,AdjClose_above_AdjCloseSMA10_-18,AdjClose_above_AdjCloseSMA10_-17,AdjClose_above_AdjCloseSMA10_-16,AdjClose_above_AdjCloseSMA10_-15,AdjClose_above_AdjCloseSMA10_-14,AdjClose_above_AdjCloseSMA10_-13,AdjClose_above_AdjCloseSMA10_-12,AdjClose_above_AdjCloseSMA10_-11,AdjClose_above_AdjCloseSMA10_-10,AdjClose_above_AdjCloseSMA10_-9,AdjClose_above_AdjCloseSMA10_-8,AdjClose_above_AdjCloseSMA10_-7,AdjClose_above_AdjCloseSMA10_-6,AdjClose_above_AdjCloseSMA10_-5,AdjClose_above_AdjCloseSMA10_-4,AdjClose_above_AdjCloseSMA10_-3,AdjClose_above_AdjCloseSMA10_-2,AdjClose_above_AdjCloseSMA10_-1,AdjClose_above_AdjCloseSMA10_0,AdjClose_crossed_AdjCloseSMA10_-19,AdjClose_crossed_AdjCloseSMA10_-18,AdjClose_crossed_AdjCloseSMA10

In [10]:
X_test.head()

,AdjVolume_-19,AdjVolume_-18,AdjVolume_-17,AdjVolume_-16,AdjVolume_-15,AdjVolume_-14,AdjVolume_-13,AdjVolume_-12,AdjVolume_-11,AdjVolume_-10,AdjVolume_-9,AdjVolume_-8,AdjVolume_-7,AdjVolume_-6,AdjVolume_-5,AdjVolume_-4,AdjVolume_-3,AdjVolume_-2,AdjVolume_-1,AdjVolume_0,AdjOpen_-19,AdjOpen_-18,AdjOpen_-17,AdjOpen_-16,AdjOpen_-15,AdjOpen_-14,AdjOpen_-13,AdjOpen_-12,AdjOpen_-11,AdjOpen_-10,AdjOpen_-9,AdjOpen_-8,AdjOpen_-7,AdjOpen_-6,AdjOpen_-5,AdjOpen_-4,AdjOpen_-3,AdjOpen_-2,AdjOpen_-1,AdjOpen_0,AdjHigh_-19,AdjHigh_-18,AdjHigh_-17,AdjHigh_-16,AdjHigh_-15,AdjHigh_-14,AdjHigh_-13,AdjHigh_-12,AdjHigh_-11,AdjHigh_-10,AdjHigh_-9,AdjHigh_-8,AdjHigh_-7,AdjHigh_-6,AdjHigh_-5,AdjHigh_-4,AdjHigh_-3,AdjHigh_-2,AdjHigh_-1,AdjHigh_0,AdjLow_-19,AdjLow_-18,AdjLow_-17,AdjLow_-16,AdjLow_-15,AdjLow_-14,AdjLow_-13,AdjLow_-12,AdjLow_-11,AdjLow_-10,AdjLow_-9,AdjLow_-8,AdjLow_-7,AdjLow_-6,AdjLow_-5,AdjLow_-4,AdjLow_-3,AdjLow_-2,AdjLow_-1,AdjLow_0,AdjClose_-19,AdjClose_-18,AdjClose_-17,AdjClose_-16,AdjClose_-15,AdjClose_-14,AdjClose_-13,AdjClose_-12,AdjClose_-11,AdjClose_-10,AdjClose_-9,AdjClose_-8,AdjClose_-7,AdjClose_-6,AdjClose_-5,AdjClose_-4,AdjClose_-3,AdjClose_-2,AdjClose_-1,AdjClose_0,AdjCloseRSI14_-19,AdjCloseRSI14_-18,AdjCloseRSI14_-17,AdjCloseRSI14_-16,AdjCloseRSI14_-15,AdjCloseRSI14_-14,AdjCloseRSI14_-13,AdjCloseRSI14_-12,AdjCloseRSI14_-11,AdjCloseRSI14_-10,AdjCloseRSI14_-9,AdjCloseRSI14_-8,AdjCloseRSI14_-7,AdjCloseRSI14_-6,AdjCloseRSI14_-5,AdjCloseRSI14_-4,AdjCloseRSI14_-3,AdjCloseRSI14_-2,AdjCloseRSI14_-1,AdjCloseRSI14_0,RSI_above_80_-19,RSI_above_80_-18,RSI_above_80_-17,RSI_above_80_-16,RSI_above_80_-15,RSI_above_80_-14,RSI_above_80_-13,RSI_above_80_-12,RSI_above_80_-11,RSI_above_80_-10,RSI_above_80_-9,RSI_above_80_-8,RSI_above_80_-7,RSI_above_80_-6,RSI_above_80_-5,RSI_above_80_-4,RSI_above_80_-3,RSI_above_80_-2,RSI_above_80_-1,RSI_above_80_0,RSI_below_20_-19,RSI_below_20_-18,RSI_below_20_-17,RSI_below_20_-16,RSI_below_20_-15,RSI_below_20_-14,RSI_below_20_-13,RSI_below_20_-12,RSI_below_20_-11,RSI_below_20_-10,RSI_below_20_-9,RSI_below_20_-8,RSI_below_20_-7,RSI_below_20_-6,RSI_below_20_-5,RSI_below_20_-4,RSI_below_20_-3,RSI_below_20_-2,RSI_below_20_-1,RSI_below_20_0,AdjCloseSMA10_-19,AdjCloseSMA10_-18,AdjCloseSMA10_-17,AdjCloseSMA10_-16,AdjCloseSMA10_-15,AdjCloseSMA10_-14,AdjCloseSMA10_-13,AdjCloseSMA10_-12,AdjCloseSMA10_-11,AdjCloseSMA10_-10,AdjCloseSMA10_-9,AdjCloseSMA10_-8,AdjCloseSMA10_-7,AdjCloseSMA10_-6,AdjCloseSMA10_-5,AdjCloseSMA10_-4,AdjCloseSMA10_-3,AdjCloseSMA10_-2,AdjCloseSMA10_-1,AdjCloseSMA10_0,AdjCloseSMA10_to_AdjClose_ratio_-19,AdjCloseSMA10_to_AdjClose_ratio_-18,AdjCloseSMA10_to_AdjClose_ratio_-17,AdjCloseSMA10_to_AdjClose_ratio_-16,AdjCloseSMA10_to_AdjClose_ratio_-15,AdjCloseSMA10_to_AdjClose_ratio_-14,AdjCloseSMA10_to_AdjClose_ratio_-13,AdjCloseSMA10_to_AdjClose_ratio_-12,AdjCloseSMA10_to_AdjClose_ratio_-11,AdjCloseSMA10_to_AdjClose_ratio_-10,AdjCloseSMA10_to_AdjClose_ratio_-9,AdjCloseSMA10_to_AdjClose_ratio_-8,AdjCloseSMA10_to_AdjClose_ratio_-7,AdjCloseSMA10_to_AdjClose_ratio_-6,AdjCloseSMA10_to_AdjClose_ratio_-5,AdjCloseSMA10_to_AdjClose_ratio_-4,AdjCloseSMA10_to_AdjClose_ratio_-3,AdjCloseSMA10_to_AdjClose_ratio_-2,AdjCloseSMA10_to_AdjClose_ratio_-1,AdjCloseSMA10_to_AdjClose_ratio_0,AdjClose_above_AdjCloseSMA10_-19,AdjClose_above_AdjCloseSMA10_-18,AdjClose_above_AdjCloseSMA10_-17,AdjClose_above_AdjCloseSMA10_-16,AdjClose_above_AdjCloseSMA10_-15,AdjClose_above_AdjCloseSMA10_-14,AdjClose_above_AdjCloseSMA10_-13,AdjClose_above_AdjCloseSMA10_-12,AdjClose_above_AdjCloseSMA10_-11,AdjClose_above_AdjCloseSMA10_-10,AdjClose_above_AdjCloseSMA10_-9,AdjClose_above_AdjCloseSMA10_-8,AdjClose_above_AdjCloseSMA10_-7,AdjClose_above_AdjCloseSMA10_-6,AdjClose_above_AdjCloseSMA10_-5,AdjClose_above_AdjCloseSMA10_-4,AdjClose_above_AdjCloseSMA10_-3,AdjClose_above_AdjCloseSMA10_-2,AdjClose_above_AdjCloseSMA10_-1,AdjClose_above_AdjCloseSMA10_0,AdjClose_crossed_AdjCloseSMA10_-19,AdjClose_crossed_AdjCloseSMA10_-18,AdjClose_crossed_AdjCloseSMA10

In [11]:
y_train

Index
2013-11-13    False
2013-11-13    False
2013-11-13    False
2013-11-13     True
2013-11-13     True
2013-11-13    False
2013-11-13     True
2013-11-13    False
2013-11-14    False
2013-11-14    False
2013-11-14     True
2013-11-14    False
2013-11-14     True
2013-11-14    False
2013-11-14     True
2013-11-14    False
2013-11-15     True
2013-11-15    False
2013-11-15    False
2013-11-15    False
2013-11-15     True
2013-11-15     True
2013-11-15     True
2013-11-15    False
2013-11-18    False
2013-11-18    False
2013-11-18     True
2013-11-18     True
2013-11-18     True
2013-11-18    False
              ...  
2017-05-05    False
2017-05-05    False
2017-05-05    False
2017-05-05    False
2017-05-05    False
2017-05-05    False
2017-05-08    False
2017-05-08    False
2017-05-08    False
2017-05-08    False
2017-05-08    False
2017-05-08    False
2017-05-08    False
2017-05-08    False
2017-05-09    False
2017-05-09    False
2017-05-09    False
2017-05-09    False
2017-05-09    

## 2) Build Initial Model - RandomForestClassifier

Start the ML process with a simple out of the box RandomForestClassifier to get a base line and validate that the training data is functioning with sklearn and can generate some predictions.

In [ ]:
# define a simple pipeline
pipeline = Pipeline([
    ('standardScaler', StandardScaler()),
    ('randomForest', RandomForestClassifier())
])

# inspect parameters
pipeline.get_params()

In [13]:
# fit the pipeline with all default parameters
pipeline.fit(X_train, y_train.reset_index().setup_for_profitable_trade)

C:\Users\Lindstaedt\Anaconda3\envs\trader_env\lib\site-packages\sklearn\preprocessing\data.py:617: DataConversionWarning: Data with input dtype bool, uint8, int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\Lindstaedt\Anaconda3\envs\trader_env\lib\site-packages\sklearn\base.py:465: DataConversionWarning: Data with input dtype bool, uint8, int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
C:\Users\Lindstaedt\Anaconda3\envs\trader_env\lib\site-packages\sklearn\ensemble\forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Pipeline(memory=None,
     steps=[('standardScaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('randomForest', RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_...obs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))])

In [14]:
# create prediction
y_pred_firstRF = pipeline.predict(X_test)

# save for backtesting in separate notebook
pickle.dump(y_pred_firstRF, open('../data/model_predictions/y_pred_firstRF.pkl', 'wb'))

C:\Users\Lindstaedt\Anaconda3\envs\trader_env\lib\site-packages\sklearn\pipeline.py:331: DataConversionWarning: Data with input dtype bool, uint8, int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)


We now have a first prediction on the dataset - let's look into the performance of the default settings. To understand the result, we will look at a few different metrics: the accuracy, precision, recall, f1-score, and the confusion matrix.

It is important to note that our data is imbalanced (not the same number of days with trade signal as with no signal). Therefore, F scores are a better metric than ROC-AUC or accuracy. We can not easily fix the imbalance since we are looking at time series data and resampling might introduce lookahead bias.

The ultimate test of the quality of the prediction is to backtest the results (ie simulate financial performance based on the predictions). This will be done in the next workbook. For now let's inspect some of the basic metrics.

In [15]:
print(classification_report(y_test, y_pred_firstRF))

              precision    recall  f1-score   support

       False       0.64      0.83      0.72      1153
        True       0.22      0.09      0.13       603

   micro avg       0.58      0.58      0.58      1756
   macro avg       0.43      0.46      0.43      1756
weighted avg       0.49      0.58      0.52      1756



In [16]:
accuracy_score(y_test, y_pred_firstRF)

0.576879271070615

In [17]:
confusion_matrix(y_test, y_pred_firstRF, labels=[False, True])

array([[957, 196],
       [547,  56]], dtype=int64)

The confusion matrix indicates a large number of False Positives (bad because if we used those to trade, we would enter a trade that turns out not to be as profitable as desired) and False Negatives (bad because they mean missed opportunities for us to enter profitable trades).

The good news is that our out-of-the-box model was able to predict some True Positives but the results in terms of financial performance are most likely very bad (we saved off the prediction and will look at this later).

Even though the model seems to perform poorly, we might be able to learn something from it:

Let's look into the relative feature importances to see if anything stands out

In [24]:
def print_feature_importances(estimator):

    importances = estimator.feature_importances_
    indices = np.argsort(importances)[::-1]

    # Print the feature ranking
    print("Feature ranking:")

    for f in range(X_train.shape[1]):
        print(str(f + 1) + " importance: " + str(importances[indices[f]]) + ". feature name: " + X_train.columns[indices[f]])

In [13]:
      print_feature_importances(pipeline.named_steps['randomForest'])

Feature ranking:
1 importance: 0.00656343610664476. feature name: AdjCloseSMA50_to_AdjCloseSMA200_ratio_-13
2 importance: 0.006452878183236933. feature name: AdjCloseSMA50_to_AdjCloseSMA200_ratio_-5
3 importance: 0.006166268408603801. feature name: AdjCloseSMA50_-4
4 importance: 0.00590650747377121. feature name: AdjCloseSMA50_chg_-19
5 importance: 0.005612619325106312. feature name: AdjCloseSMA50_-17
6 importance: 0.005579566256702442. feature name: AdjCloseSMA200_to_AdjClose_ratio_-6
7 importance: 0.005569776903122497. feature name: AdjCloseSMA50_chg_-18
8 importance: 0.005452466290812995. feature name: AdjClose_-6
9 importance: 0.005443589795855236. feature name: AdjCloseSMA200_-11
10 importance: 0.005402202941807866. feature name: AdjCloseSMA10_chg_-19
11 importance: 0.005291209402120946. feature name: AdjCloseSMA50_to_AdjCloseSMA200_ratio_-15
12 importance: 0.005195837288783827. feature name: AdjCloseSMA50_to_AdjCloseSMA200_ratio_-8
13 importance: 0.005131298455312763. feature nam

217 importance: 0.0019791772853010887. feature name: AdjCloseSMA10_to_AdjClose_ratio_-18
218 importance: 0.0019702982845467122. feature name: AdjCloseSMA200_chg_-5
219 importance: 0.0019655861310386323. feature name: AdjVolume_-4
220 importance: 0.0019596351775260454. feature name: AdjCloseSMA50_-3
221 importance: 0.0019543518643614184. feature name: AdjOpen_-13
222 importance: 0.001953695966109234. feature name: AdjCloseSMA200_to_AdjClose_ratio_-19
223 importance: 0.0019418112066206163. feature name: AdjLow_-1
224 importance: 0.001936803253455316. feature name: AdjHigh_-15
225 importance: 0.001935400821418141. feature name: AdjVolume_-6
226 importance: 0.0019218608420739404. feature name: AdjLow_chg_-3
227 importance: 0.0019084682850617062. feature name: AdjCloseSMA50_to_AdjCloseSMA200_ratio_-17
228 importance: 0.001897394452344392. feature name: AdjCloseSMA200_chg_-3
229 importance: 0.0018941637209664956. feature name: AdjHigh_-18
230 importance: 0.0018914332159692833. feature name: 

444 importance: 0.00047833854007584356. feature name: AdjCloseSMA50_to_AdjClose_ratio_-1
445 importance: 0.0004689164186528853. feature name: AdjCloseSMA200_to_AdjClose_ratio_-13
446 importance: 0.00045834805074172367. feature name: AdjCloseSMA200_above_AdjCloseSMA50_-10
447 importance: 0.00043886664531333185. feature name: AdjOpen_-16
448 importance: 0.00042536376437622345. feature name: AdjHigh_-14
449 importance: 0.00042391362595498017. feature name: AAPL.US
450 importance: 0.0004010679975969036. feature name: AdjCloseSMA50_above_AdjCloseSMA10_-3
451 importance: 0.0003879367355047792. feature name: AdjClose_crossed_AdjCloseSMA10_-6
452 importance: 0.00037853536350914455. feature name: AdjCloseSMA200_above_AdjCloseSMA50_-3
453 importance: 0.00036034631932015885. feature name: AdjCloseSMA200_above_AdjCloseSMA50_-1
454 importance: 0.00035437260511606694. feature name: AdjCloseSMA10_-15
455 importance: 0.0003451118465427925. feature name: AdjClose_crossed_AdjCloseSMA10_-10
456 importanc

649 importance: 0.0. feature name: AdjClose_above_AdjCloseSMA50_-1
650 importance: 0.0. feature name: AdjClose_crossed_AdjCloseSMA50_-19
651 importance: 0.0. feature name: RSI_below_20_-9
652 importance: 0.0. feature name: RSI_below_20_-10
653 importance: 0.0. feature name: RSI_below_20_-11
654 importance: 0.0. feature name: RSI_above_80_-17
655 importance: 0.0. feature name: RSI_above_80_-19
656 importance: 0.0. feature name: AdjClose_above_AdjCloseSMA200_-7
657 importance: 0.0. feature name: AdjClose_above_AdjCloseSMA200_-6
658 importance: 0.0. feature name: AdjClose_above_AdjCloseSMA200_-5
659 importance: 0.0. feature name: AdjClose_above_AdjCloseSMA200_-4
660 importance: 0.0. feature name: AdjClose_above_AdjCloseSMA200_-3
661 importance: 0.0. feature name: AdjClose_above_AdjCloseSMA200_-1
662 importance: 0.0. feature name: AdjClose_above_AdjCloseSMA200_0
663 importance: 0.0. feature name: AdjClose_crossed_AdjCloseSMA200_-17
664 importance: 0.0. feature name: AdjClose_crossed_AdjClo

At the very bottom of the list are the RSI_above_80 and RSI_below_20 indicators. RSI is an indicator of extremes so having these features not play an important role means that the model did not notice many extremes and having predictive power.
The top section contains many of the SMA50 to SMA200 ratios. The SMA 50 and 200 crossovers or distances are classic momentum indicators (SMA 50 over SMA 200 = uptrend, below = downtrend).
Since the time period in the training data is that of a general market uptrend, it sounds intuitively reasonable that momentum indicators are important.

## 3) Improve RandomForest model with GridSearch

Given the imbalance of our data and the desire to avoid False Positives, we will use a modified F1 score that places stronger emphasis on precision than on recall (they have the same weight in the F1 score). We will skew it towards precision by setting beta to 0.5.

In [18]:
# place higher focus on precision (ie getting most TP and minimize FP) than on recall (ie minimize FN)
# since placing a trade that was based on a FP will be costly and hurts more than missing a trade due to a FN
fhalf_scorer = make_scorer(fbeta_score, beta=0.5) # low beta favors precision over recall

For a GridSearch, we will define a wide range of parameters to be used in the pipeline. The new fhalf_scorer will be used for optimization.

In [ ]:
parameters = {
    'randomForest__min_samples_leaf': [2, 5, 10],
    'randomForest__n_estimators' : [10, 20, 50, 100],
    'randomForest__max_features': [5, 'sqrt', 'log2'], # log2 of 690 = 9, sqrt of 690 = 20
    'randomForest__max_depth' : [4, 5, 6, 7, 8],
    'randomForest__criterion' :['gini', 'entropy']
}

cv = GridSearchCV(pipeline, param_grid=parameters, cv=5, scoring=fhalf_scorer, n_jobs=-1, verbose=10) # uses fhalf_scorer
cv.fit(X_train, y_train)

Fitting 5 folds for each of 360 candidates, totalling 1800 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:   20.6s
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:   28.5s
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:   49.5s
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:   59.5s
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done  53 tasks      | elapsed:  2.3min
[Parallel(n_jobs=-1)]: Done  64 tasks      | elapsed:  2.9min
[Parallel(n_jobs=-1)]: Done  77 tasks      | elapsed:  4.0min
[Parallel(n_jobs=-1)]: Done  90 tasks      | elapsed:  4.6min
[Parallel(n_jobs=-1)]: Done 105 tasks      | elapsed:  5.6min
[Parallel(n_jobs=-1)]: Done 120 tasks      | elapsed:  6.6min
[Parallel(n_jobs=-1)]: Done 137 tasks      | elapsed:  7.4min
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:  8.1min
[Parallel(n_jobs=-1)]: Done 173 tasks      | elapsed:  9

In [22]:
cv.best_params_

{'randomForest__min_samples_leaf': 2, 'randomForest__n_estimators': 10}

In [25]:
print_feature_importances(cv.best_estimator_.named_steps['randomForest'])

Feature ranking:
1 importance: 0.00790016740540973. feature name: week
2 importance: 0.007609707364517186. feature name: AdjCloseSMA50_chg_-15
3 importance: 0.006642301872023167. feature name: AdjCloseSMA50_chg_-16
4 importance: 0.0059754609629070875. feature name: AdjCloseSMA50_chg_-14
5 importance: 0.005776145390214105. feature name: AdjCloseSMA200_-19
6 importance: 0.005755064826525115. feature name: AdjCloseSMA50_to_AdjCloseSMA200_ratio_-5
7 importance: 0.005687847439591708. feature name: AdjCloseSMA200_to_AdjClose_ratio_-19
8 importance: 0.005669924884718845. feature name: AdjCloseSMA50_to_AdjCloseSMA200_ratio_-4
9 importance: 0.005331111637959928. feature name: AdjCloseSMA200_chg_0
10 importance: 0.005329092182894403. feature name: AdjCloseSMA200_-14
11 importance: 0.005316884480539015. feature name: AdjCloseSMA200_to_AdjClose_ratio_-14
12 importance: 0.0052148075518322285. feature name: AdjCloseSMA50_to_AdjCloseSMA200_ratio_-2
13 importance: 0.005130356799173274. feature name: A

105 importance: 0.002831964243356755. feature name: AdjCloseSMA50_-14
106 importance: 0.002805300719279668. feature name: AdjCloseSMA10_chg_-12
107 importance: 0.0028020837742868117. feature name: AdjClose_chg_-18
108 importance: 0.0027966270935461577. feature name: AdjHigh_chg_-11
109 importance: 0.002784430056650472. feature name: AdjLow_chg_-16
110 importance: 0.00278113660230746. feature name: day
111 importance: 0.0027431626674878353. feature name: AdjCloseSMA50_to_AdjClose_ratio_-18
112 importance: 0.002739556194955543. feature name: AdjCloseSMA200_chg_-12
113 importance: 0.0027014331972181104. feature name: AdjCloseSMA50_-10
114 importance: 0.002695621120722013. feature name: AdjCloseSMA10_to_AdjClose_ratio_-18
115 importance: 0.0026694973128885456. feature name: AdjCloseSMA10_to_AdjCloseSMA50_ratio_-9
116 importance: 0.0026691279304374197. feature name: AdjCloseSMA10_to_AdjCloseSMA50_ratio_-6
117 importance: 0.002665038992400458. feature name: AdjCloseSMA50_to_AdjClose_ratio_-1

216 importance: 0.00206161184108835. feature name: AdjVolume_chg_-4
217 importance: 0.0020581945968876935. feature name: AdjVolume_-19
218 importance: 0.0020562171877255555. feature name: AdjCloseRSI14_-8
219 importance: 0.0020459786308710667. feature name: AdjCloseRSI14_-14
220 importance: 0.0020459103156164633. feature name: AdjCloseSMA50_-3
221 importance: 0.0020453152713150687. feature name: AdjOpen_chg_-7
222 importance: 0.002040094964730835. feature name: AdjCloseSMA200_to_AdjClose_ratio_-4
223 importance: 0.002035780668855684. feature name: AdjCloseSMA200_chg_-3
224 importance: 0.0020347666479042467. feature name: AdjCloseSMA50_chg_-18
225 importance: 0.002032822706910635. feature name: AdjCloseRSI14_-15
226 importance: 0.0020300455498061093. feature name: AdjCloseSMA50_-11
227 importance: 0.002019259571318945. feature name: AdjHigh_-17
228 importance: 0.0020183078757308316. feature name: AdjCloseSMA50_to_AdjClose_ratio_-4
229 importance: 0.002015677235707294. feature name: AdjC

330 importance: 0.001437946947093523. feature name: AdjCloseSMA10_to_AdjClose_ratio_-2
331 importance: 0.0014333253907972768. feature name: AdjCloseSMA10_to_AdjClose_ratio_-4
332 importance: 0.001427429270222513. feature name: AdjLow_chg_-15
333 importance: 0.001421975003547169. feature name: AdjHigh_chg_-1
334 importance: 0.001418562081481859. feature name: AdjVolume_-18
335 importance: 0.0014146221223204837. feature name: AdjHigh_-16
336 importance: 0.0014054634011733665. feature name: AdjOpen_-14
337 importance: 0.0013996666599027556. feature name: AdjVolume_-5
338 importance: 0.0013879494214106718. feature name: AdjClose_-9
339 importance: 0.001375093557231834. feature name: AdjCloseRSI14_-13
340 importance: 0.0013702726063278044. feature name: AdjOpen_-11
341 importance: 0.0013693439973102852. feature name: AdjVolume_chg_-15
342 importance: 0.0013655172380800291. feature name: AdjCloseSMA200_to_AdjClose_ratio_-9
343 importance: 0.0013560214237659006. feature name: AdjCloseSMA10_to

444 importance: 0.000493969825285415. feature name: RSI_above_80_-9
445 importance: 0.0004519533603931129. feature name: AMZN.US
446 importance: 0.00042453405215962255. feature name: AdjClose_crossed_AdjCloseSMA10_-17
447 importance: 0.00042149579852426237. feature name: AdjHigh_-6
448 importance: 0.0004124287346316573. feature name: AdjClose_crossed_AdjCloseSMA10_-19
449 importance: 0.00040728798035968. feature name: AdjClose_-19
450 importance: 0.0004038980983668872. feature name: AdjClose_crossed_AdjCloseSMA10_-3
451 importance: 0.00038569145102753934. feature name: AdjClose_-10
452 importance: 0.0003694491719451698. feature name: AdjClose_above_AdjCloseSMA10_0
453 importance: 0.00036746415962924376. feature name: AdjClose_crossed_AdjCloseSMA10_-15
454 importance: 0.000356344479534164. feature name: AdjOpen_chg_-2
455 importance: 0.0003466570107723577. feature name: AdjClose_crossed_AdjCloseSMA10_-14
456 importance: 0.00034184209187747966. feature name: AdjCloseSMA10_-4
457 importan

538 importance: 2.761801985678786e-05. feature name: AdjClose_crossed_AdjCloseSMA200_-10
539 importance: 2.6792329048038654e-05. feature name: AdjCloseSMA50_crossed_AdjCloseSMA10_0
540 importance: 2.6683024120762643e-05. feature name: AdjCloseSMA200_crossed_AdjCloseSMA50_-7
541 importance: 2.2358250928859748e-05. feature name: AdjClose_crossed_AdjCloseSMA200_-6
542 importance: 2.2329392188337134e-05. feature name: AdjCloseSMA50_above_AdjCloseSMA10_-18
543 importance: 2.1884706301175182e-05. feature name: AdjClose_crossed_AdjCloseSMA200_-13
544 importance: 2.0826013599616262e-05. feature name: RSI_above_80_-10
545 importance: 9.499213316419115e-06. feature name: AdjClose_crossed_AdjCloseSMA50_-19
546 importance: 0.0. feature name: AdjClose_crossed_AdjCloseSMA50_-14
547 importance: 0.0. feature name: AdjClose_crossed_AdjCloseSMA50_0
548 importance: 0.0. feature name: AdjCloseSMA200_crossed_AdjCloseSMA50_-15
549 importance: 0.0. feature name: AdjCloseSMA200_crossed_AdjCloseSMA50_-17
550 i

655 importance: 0.0. feature name: RSI_above_80_-16
656 importance: 0.0. feature name: RSI_above_80_-17
657 importance: 0.0. feature name: RSI_above_80_-18
658 importance: 0.0. feature name: RSI_above_80_-19
659 importance: 0.0. feature name: AdjClose_above_AdjCloseSMA50_-15
660 importance: 0.0. feature name: RSI_below_20_-3
661 importance: 0.0. feature name: AdjClose_above_AdjCloseSMA50_-13
662 importance: 0.0. feature name: AdjClose_above_AdjCloseSMA50_-12
663 importance: 0.0. feature name: AdjClose_above_AdjCloseSMA50_-10
664 importance: 0.0. feature name: AdjClose_above_AdjCloseSMA50_-8
665 importance: 0.0. feature name: AdjClose_above_AdjCloseSMA50_-7
666 importance: 0.0. feature name: AdjClose_above_AdjCloseSMA50_-6
667 importance: 0.0. feature name: AdjClose_above_AdjCloseSMA50_-5
668 importance: 0.0. feature name: AdjClose_above_AdjCloseSMA50_-3
669 importance: 0.0. feature name: AdjClose_0
670 importance: 0.0. feature name: RSI_above_80_-7
671 importance: 0.0. feature name: RS

In [26]:
y_pred_GridSearch = cv.predict(X_test)

# save for backtesting in separate notebook
pickle.dump(y_pred_GridSearch, open('../data/model_predictions/y_pred_GridSearch.pkl', 'wb'))

C:\Users\Lindstaedt\Anaconda3\envs\trader_env\lib\site-packages\sklearn\pipeline.py:331: DataConversionWarning: Data with input dtype bool, uint8, int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)


In [27]:
print(classification_report(y_test, y_pred_GridSearch))

              precision    recall  f1-score   support

       False       0.64      0.77      0.70      1153
        True       0.28      0.17      0.21       603

   micro avg       0.56      0.56      0.56      1756
   macro avg       0.46      0.47      0.45      1756
weighted avg       0.52      0.56      0.53      1756



In [28]:
accuracy_score(y_test, y_pred_GridSearch)

0.5620728929384966

In [29]:
confusion_matrix(y_test, y_pred_GridSearch, labels=[False, True])

array([[883, 270],
       [499, 104]], dtype=int64)

## next: resume in backtesting notebook